In [1]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
import $ivy.`org.apache.spark::spark-mllib:2.3.0`
import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

Checking https://repo1.maven.org/maven2/joda-time/joda-time/maven-metadata.xml
Checking https://repo1.maven.org/maven2/joda-time/joda-time/maven-metadata.xml.sha1
Checked https://repo1.maven.org/maven2/joda-time/joda-time/maven-metadata.xml.sha1
Checked https://repo1.maven.org/maven2/joda-time/joda-time/maven-metadata.xml


import $exclude.$                        , $ivy.$                            // for cleaner logs

import $profile.$           

import $ivy.$                                   // adjust spark version - spark >= 2.0

import $ivy.$                                    

import $ivy.$                                   

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

In [2]:
import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._

import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._

In [3]:
val sparkSession = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  // .yarn("/etc/hadoop/conf") // optional, for Spark on YARN - argument is the Hadoop conf directory
  // .emr("2.6.4") // on AWS ElasticMapReduce, this adds aws-related to the spark jar list
  .master("local") // change to "yarn-client" on YARN
  // .config("spark.executor.instances", "10")
  // .config("spark.executor.memory", "3g")
  // .config("spark.hadoop.fs.s3a.access.key", awsCredentials._1)
  // .config("spark.hadoop.fs.s3a.secret.key", awsCredentials._2)
  .appName("notebook")
  .getOrCreate()

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


sparkSession: SparkSession = org.apache.spark.sql.SparkSession@26400338

In [4]:
var train = sparkSession.read.parquet("/home/mahjoubi/Documents/github/toxic_comment/data/parquet/train")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


train: DataFrame = [id: string, comment_text: string ... 6 more fields]

In [17]:
val firstRow = Seq(train.first())
val data = sparkSession.sparkContext.parallelize(firstRow)
data.collect()

firstRow: Seq[Row] = List(
  [9f14426bcd93c0f8,"

 Edit warring 

After the edit warring on this page, I have protected it for a week and started a discussion at Wikipedia:Redirects_for_discussion/Log/2013_January_16#Halifax.2C_Nova_Scotia.  The results of that discussion should be respected, and no further edit warring should take place.  |Â communicateÂ _ ",0,0,0,0,0,0]
)
data: rdd.RDD[Row] = ParallelCollectionRDD[34] at parallelize at cmd16.sc:2
res16_2: Array[Row] = Array(
  [9f14426bcd93c0f8,"

 Edit warring 

After the edit warring on this page, I have protected it for a week and started a discussion at Wikipedia:Redirects_for_discussion/Log/2013_January_16#Halifax.2C_Nova_Scotia.  The results of that discussion should be respected, and no further edit warring should take place.  |Â communicateÂ _ ",0,0,0,0,0,0]
)

In [22]:
import org.apache.spark.sql.types.StructField
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.types.LongType
import org.apache.spark.sql.types.StringType

val schema = List(StructField("id", StringType, true),
StructField("comment_text", StringType, true),
StructField("toxic", LongType, true),
StructField("severe_toxic", LongType, true),
StructField("obscene", LongType, true),
StructField("threat", LongType, true),
StructField("insult", LongType, true),
StructField("identity_hate", LongType, true))

import org.apache.spark.sql.types.StructField

import org.apache.spark.sql.types.StructType

import org.apache.spark.sql.types.LongType

import org.apache.spark.sql.types.StringType


schema: List[StructField] = List(
  StructField(id,StringType,true),
  StructField(comment_text,StringType,true),
  StructField(toxic,LongType,true),
  StructField(severe_toxic,LongType,true),
  StructField(obscene,LongType,true),
  StructField(threat,LongType,true),
  StructField(insult,LongType,true),
  StructField(identity_hate,LongType,true)
)

In [23]:
val dataSet = sparkSession.createDataFrame(data, StructType(schema))

dataSet: DataFrame = [id: string, comment_text: string ... 6 more fields]

In [18]:
dataSet.show()

someData: Seq[Row] = List([8,bat], [64,mouse], [-27,horse])

In [24]:
dataSet.coalesce(1).write.parquet("/home/mahjoubi/Documents/github/toxic_comment/src/test/ressources/data/train")